In [1]:
from Voxel_Data_Process import *
import gzip,pickle
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc

from scipy import interp
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


In [2]:
def read_y_val(pixel_size, fold):
#     (x_train,y_train,x_val,y_val)=load_fold_data(pixel_size,fold)
#     (x_train,y_train,x_val,y_val)=k_fold(pixel_size,fold)
    (x_train,y_train,x_val,y_val)=k_fold(pixel_size,fold,0)

    return y_val


# 将y_val从[0 1]的双标签转为单标签
def convert_label(y_val):
     
    y_val_after = []
    y_val = list(y_val)
    
    for t in y_val:
        t = list(t)       
        if t == [1,0]:
            y_val_after.append(0)
        else:
            y_val_after.append(1)
    
    return (np.array(y_val_after))

def convert_label_pred(y_val_pred):
    # 阈值
    threshold = 0.5
    
    predicted_class = []
    
    for temp in y_val_pred:
        if temp[1] < threshold:
            predicted_class.append(0)
        else:
            predicted_class.append(1)
        
    return (np.array(predicted_class))


In [3]:
def read_y_val_pred(function, time, ROC):
    
    # 预测的y_val_pred, 0-4对于第几则训练
    y_val_pred = []
    
    # 判断是否含有SVM
    if ('SVM' in function):
        function1 = function[:-4]
        
        for fold in range(1,6):
            with gzip.open('./Predict/'+function+'/'+function+'_Pred_'+(str)(fold)+'_4.pkl','rb') as f:
                temp = pickle.load(f)
                if ROC == 0:
                    temp = convert_label_pred(temp)
                y_val_pred.append(temp)
        print ('SVM')
    else:
        for fold in range(1,6):
            with gzip.open('./Predict/'+function+'/'+function+'_Pred_'+(str)(fold)+'_4.pkl','rb') as f:
                temp = pickle.load(f)
                print (temp.shape)
                if ROC == 0:
                    temp = convert_label_pred(temp)
                y_val_pred.append(temp)
        print ('No, SVM')
        
        
    return y_val_pred

In [5]:
#caculate the sensitivity,
def ARSS(function, time):
    
    # 下标0-4对于第几则训练，5为平均值
    accuracy = []
    sensitivity = []
    specificity = []
    recall = []
    # 预测的y_val_pred
    y_val_pred = read_y_val_pred(function, time, 0)
    
        
    # 初始化 False Positive Rate和True Positive Rate
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    
    for fold in range(0, 5):
        
        
        cm1 = confusion_matrix(y_val[fold], y_val_pred[fold])
        total1=sum(sum(cm1))
        
        # 计算recall        
        recall1 = recall_score(y_val[fold], y_val_pred[fold], average='weighted')
        recall.append(recall1)
        #####from confusion matrix calculate accuracy
        accuracy1=float((cm1[0,0]+cm1[1,1]))/float(total1)
        accuracy.append(accuracy1)
         
        sensitivity1 = float(cm1[0,0])/float((cm1[0,0]+cm1[0,1]))
        sensitivity.append(sensitivity1)
        
        specificity1 = float(cm1[1,1])/float((cm1[1,0]+cm1[1,1]))
        specificity.append(specificity1)
        

    # 计算平均值
    accuracy_mean = (accuracy[0]+accuracy[1]+accuracy[2]+accuracy[3]+accuracy[4])/5
    print (accuracy)
    sensitivity_mean = (sensitivity[0]+sensitivity[1]+sensitivity[2]+sensitivity[3]+sensitivity[4])/5
    specificity_mean = (specificity[0]+specificity[1]+specificity[2]+specificity[3]+specificity[4])/5
    recall_mean = (recall[0]+recall[1]+recall[2]+recall[3]+recall[4])/5
    accuracy.append(accuracy_mean)
    sensitivity.append(sensitivity_mean)
    specificity.append(specificity_mean)
    recall.append(recall_mean)
    
    
    print ('======== '+function+' ========')
    print ('Accuracy : '+str(accuracy[5]))
    print ('Sensitivity : '+str(sensitivity[5]))
    print ('Specificity : '+str(specificity[5]))
    print ('Recall : '+str(recall[5]))
    
    
#     f = open('/home/ilab/Jupyter/CNN/Voxel_PY/Result/'+function+'.txt','w')
#     f.write('======== '+function+' ========')
#     f.write('\n')
#     f.write('Accuracy : '+str(accuracy[5])+'\n')
#     f.write('Sensitivity : '+str(sensitivity[5])+'\n')
#     f.write('Specificity : '+str(specificity[5])+'\n')
#     f.write('Recall : '+str(recall[5])+'\n')
    
#     f.close()
       

In [4]:
y_val = []
for fold in range(1, 6):
    temp = read_y_val(pixel_size, fold)
    temp = convert_label(temp)
    y_val.append(temp)

397
384
(2343, 64, 64, 64, 1)
Start to split dataset....
Random split dataset success....
397
384
(2343, 64, 64, 64, 1)
Start to split dataset....
Random split dataset success....
397
384
(2343, 64, 64, 64, 1)
Start to split dataset....
Random split dataset success....
397
384
(2343, 64, 64, 64, 1)
Start to split dataset....
Random split dataset success....
397
384
(2343, 64, 64, 64, 1)
Start to split dataset....
Random split dataset success....


In [9]:
# ARSS('C3D_Patience15', 1)
ARSS('Final_Sailency_ConMultiply_FeatureAddFusion_Addloss_Patience15', 4)
ARSS('Final_NO_Saliency_SSD_Patience15',4)

(469, 2)
(469, 2)
(469, 2)
(469, 2)
(469, 2)
No, SVM
[0.9424307036247335, 0.9360341151385928, 0.9402985074626866, 0.9488272921108742, 0.9466950959488273]
======== Final_Sailency_ConMultiply_FeatureAddFusion_Addloss_Patience15 ========
Accuracy : 0.942857142857
Sensitivity : 0.944583921378
Specificity : 0.936994053721
Recall : 0.9428571428571428
(469, 2)
(469, 2)
(469, 2)
(469, 2)
(469, 2)
No, SVM
[0.835820895522388, 0.8294243070362474, 0.8294243070362474, 0.8805970149253731, 0.8763326226012793]
======== Final_NO_Saliency_SSD_Patience15 ========
Accuracy : 0.850319829424
Sensitivity : 0.837738752696
Specificity : 0.881247552026
Recall : 0.8503198294243071
